In [1]:
import requests
import json
import bs4
import pandas as pd
import datetime
import sqlite3
import ast
import warnings

from geopy.geocoders import Nominatim
from datetime import datetime

warnings.filterwarnings('ignore')

In [2]:
import geopandas as gpd
import psycopg2
from shapely.geometry import Point, LineString, Polygon

In [3]:
conn = psycopg2.connect(dbname="postgis", 
                 user="gsa2022", 
                 password="g5!V%T1Vmd", 
                 host="192.168.212.99", 
                 port=32771)

# Goals
If there are more amenities in an area, it could mean more damage in the infrastructures when a strong earthquake strikes. This notebook would focus on data preparation to be used in the analysis. Number of hospitals, schools and the top 20 amenities by counts will be explored per municipality.


## Amenities count of all affected areas

1. Number of hospitals
2. Number of schools
3. All top 20 amenities

In [4]:
df_new= pd.read_csv('cleaned_affected_areas_eq.csv')

In [5]:
df_new.head(2)

,Unnamed: 0,name_0,name_1,name_2,geom,point,coordinates,mag,place,time,...,magType,coordinates.1,longitude,latitude,Point,light,moderate,strong,major,year
0,0,Philippines,Abra,Bucay,"MULTIPOLYGON (((120.70787811 17.45395088, 120....",0101000020330C0000FE0B4B8FA0C71C41BD200D81D2BD...,"[125.704, 6.176, 122.7]",4.6,"3 km N of Lapuan, Philippines",2012-07-29 03:27:50.000,...,mb,"[125.704, 6.176, 122.7]",125.704,6.176,POINT (125.704 6.176),1,0,0,0,2012
1,1,Philippines,Abra,Dolores,"MULTIPOLYGON (((120.70375061 17.62532997, 120....",0101000020330C0000FE0B4B8FA0C71C41BD200D81D2BD...,"[125.704, 6.176, 122.7]",4.6,"3 km N of Lapuan, Philippines",2012-07-29 03:27:50.000,...,mb,"[125.704, 6.176, 122.7]",125.704,6.176,POINT (125.704 6.176),1,0,0,0,2012


In [6]:
#Code Here
df_hospital = pd.read_sql("""

select g.name_1, g.name_2,
       count(*) as hospital_count
from gadm.ph as g
join public.ph_point as p
on ST_Within(p.way, g.geom)
and p.amenity = 'hospital'
group by name_1, name_2
""", conn)


In [7]:
df_hospital

,name_1,name_2,hospital_count
0,Agusan del Norte,Butuan City,4
1,Agusan del Norte,Cabadbaran City,1
2,Agusan del Sur,Bayugan City,3
3,Agusan del Sur,Bunawan,1
4,Agusan del Sur,Esperanza,1
...,...,...,...
352,Zambales,Santa Cruz,1
353,Zambales,Subic,2
354,Zamboanga del Norte,Dipolog City,4
355,Zamboanga del Sur,Pagadian City,4


In [8]:
#Code Here
df_school = pd.read_sql("""

select g.name_1, g.name_2,
       count(*) as school_count
from gadm.ph as g
join public.ph_point as p
on ST_Within(p.way, g.geom)
and p.amenity = 'school'
group by name_1, name_2
""", conn)


In [9]:
df_school

,name_1,name_2,school_count
0,Occidental Mindoro,Paluan,1
1,Capiz,Mambusao,3
2,Ilocos Sur,Sinait,5
3,Davao Oriental,Governor Generoso,1
4,Northern Samar,Catarman,10
...,...,...,...
978,Bohol,Loboc,3
979,Capiz,Dumarao,2
980,Batangas,Lobo,1
981,Negros Oriental,Bais City,1


In [10]:
pd.read_sql("""

select amenity, count(*) as counts
from public.ph_point
where amenity is not null
group by 1
order by 2 desc
limit 10
""", con=conn)

,amenity,counts
0,restaurant,8869
1,school,5853
2,bank,5369
3,fast_food,5249
4,place_of_worship,4652
5,fuel,3799
6,pharmacy,2956
7,cafe,2171
8,parking,1661
9,bar,1177


In [11]:
df_new.name_1.nunique()

79

In [60]:
#Counts of all amenities
df_all = gpd.read_postgis("""
select g.name_1, g.name_2, geom, count(*) as amenity_counts
from gadm.ph as g
join public.ph_point as p
on ST_Within(p.way, g.geom)
group by name_1, name_2, geom
""", conn)


In [61]:
df_all.shape

(1633, 4)

In [62]:
data1 = df_all.merge(df_school, on=['name_1', 'name_2'], how='left')
data2 = data1.merge(df_hospital, on=['name_1', 'name_2'], how='left')
data2.head()

,name_1,name_2,geom,amenity_counts,school_count,hospital_count
0,Abra,Bangued,"MULTIPOLYGON (((120.62710 17.49300, 120.62177 ...",44,2.0,NaN
1,Abra,Boliney,"MULTIPOLYGON (((120.99042 17.39802, 120.98631 ...",9,NaN,NaN
2,Abra,Bucay,"MULTIPOLYGON (((120.70788 17.45395, 120.70621 ...",3,NaN,NaN
3,Abra,Bucloc,"MULTIPOLYGON (((120.78956 17.41699, 120.78922 ...",6,NaN,NaN
4,Abra,Daguioman,"MULTIPOLYGON (((120.92917 17.41307, 120.92464 ...",10,NaN,NaN


In [64]:
df_new.columns

Index(['Unnamed: 0', 'name_0', 'name_1', 'name_2', 'geom', 'point',
       'coordinates', 'mag', 'place', 'time', 'updated', 'felt', 'cdi', 'mmi',
       'alert', 'sig', 'magType', 'coordinates.1', 'longitude', 'latitude',
       'Point', 'light', 'moderate', 'strong', 'major', 'year'],
      dtype='object')

In [65]:
data2.fillna(0)

,name_1,name_2,geom,amenity_counts,school_count,hospital_count
0,Abra,Bangued,"MULTIPOLYGON (((120.62710 17.49300, 120.62177 ...",44,2.0,0.0
1,Abra,Boliney,"MULTIPOLYGON (((120.99042 17.39802, 120.98631 ...",9,0.0,0.0
2,Abra,Bucay,"MULTIPOLYGON (((120.70788 17.45395, 120.70621 ...",3,0.0,0.0
3,Abra,Bucloc,"MULTIPOLYGON (((120.78956 17.41699, 120.78922 ...",6,0.0,0.0
4,Abra,Daguioman,"MULTIPOLYGON (((120.92917 17.41307, 120.92464 ...",10,0.0,0.0
...,...,...,...,...,...,...
1628,Zamboanga Sibugay,Roseller Lim,"MULTIPOLYGON (((122.54519 7.68240, 122.54472 7...",14,0.0,0.0
1629,Zamboanga Sibugay,Siay,"MULTIPOLYGON (((122.81555 7.72806, 122.81500 7...",2,0.0,0.0
1630,Zamboanga Sibugay,Talusan,"MULTIPOLYGON (((122.91917 7.34111, 122.91861 7...",1,0.0,0.0
1631,Zamboanga Sibugay,Titay,"MULTIPOLYGON (((122.59055 7.82854, 122.58292 7...",5,0.0,0.0


In [66]:
finance_df= pd.read_csv("financial_pop.csv")
finance_df.head()

,pop,tot_local_sources,tot_tax_revenue,tot_current_oper_income,total_oper_expenses,net_oper_income,total_non_income_receipts,capital_expenditure,total_non_oper_expenditures,cash_balance_end,shp_province,shp_municipality
0,48163.0,74.045920,36.556294,230.577076,206.848717,23.728359,0.0,16.202464,21.181116,15.758681,Abra,Bangued
1,3573.0,0.115667,0.035633,56.689883,54.409910,2.279973,0.0,0.000000,0.030000,20.369743,Abra,Boliney
2,17115.0,1.736411,1.019565,93.647242,66.222389,27.424853,0.0,19.560034,20.792182,6.632671,Abra,Bucay
3,2501.0,0.273689,0.129150,44.789104,29.633181,15.155923,0.0,0.479569,0.656569,24.348356,Abra,Bucloc
4,2088.0,0.643801,0.277660,51.242322,47.848486,3.393836,0.0,0.000000,0.000000,6.511575,Abra,Daguioman


In [67]:
finance_df.columns

Index(['pop', 'tot_local_sources', 'tot_tax_revenue',
       'tot_current_oper_income', 'total_oper_expenses', 'net_oper_income',
       'total_non_income_receipts', 'capital_expenditure',
       'total_non_oper_expenditures', 'cash_balance_end', 'shp_province',
       'shp_municipality'],
      dtype='object')

In [68]:
finance_df.shape

(1627, 12)

In [69]:
#Lets merge the population column to our data 
data = pd.merge(data2, finance_df[['pop','tot_tax_revenue',
                                'tot_current_oper_income', 'shp_province',
                                 'shp_municipality']],
                     how='left',
                     left_on=['name_1', 'name_2'],
                     right_on = ['shp_province',
                               'shp_municipality']
                    )
dataset = data.fillna(0)
dataset

,name_1,name_2,geom,amenity_counts,school_count,hospital_count,pop,tot_tax_revenue,tot_current_oper_income,shp_province,shp_municipality
0,Abra,Bangued,"MULTIPOLYGON (((120.62710 17.49300, 120.62177 ...",44,2.0,0.0,48163.0,36.556294,230.577076,Abra,Bangued
1,Abra,Boliney,"MULTIPOLYGON (((120.99042 17.39802, 120.98631 ...",9,0.0,0.0,3573.0,0.035633,56.689883,Abra,Boliney
2,Abra,Bucay,"MULTIPOLYGON (((120.70788 17.45395, 120.70621 ...",3,0.0,0.0,17115.0,1.019565,93.647242,Abra,Bucay
3,Abra,Bucloc,"MULTIPOLYGON (((120.78956 17.41699, 120.78922 ...",6,0.0,0.0,2501.0,0.129150,44.789104,Abra,Bucloc
4,Abra,Daguioman,"MULTIPOLYGON (((120.92917 17.41307, 120.92464 ...",10,0.0,0.0,2088.0,0.277660,51.242322,Abra,Daguioman
...,...,...,...,...,...,...,...,...,...,...,...
1628,Zamboanga Sibugay,Roseller Lim,"MULTIPOLYGON (((122.54519 7.68240, 122.54472 7...",14,0.0,0.0,43646.0,2.543219,121.506087,Zamboanga Sibugay,Roseller Lim
1629,Zamboanga Sibugay,Siay,"MULTIPOLYGON (((122.81555 7.72806, 122.81500 7...",2,0.0,0.0,41572.0,2.042040,124.253050,Zamboanga Sibugay,Siay
1630,Zamboanga Sibugay,Talusan,"MULTIPOLYGON (((122.91917 7.34111, 122.91861 7...",1,0.0,0.0,29969.0,0.395521,67.680146,Zamboanga Sibugay,Talusan
1631,Zamboanga Sibugay,Titay,"MULTIPOLYGON (((122.59055 7.82854, 122.58292 7...",5,0.0,0.0,49673.0,3.092784,155.822081,Zamboanga Sibugay,Titay


In [70]:
data.drop(columns=['shp_province','shp_municipality'], axis=1, inplace=True)
data.head()

,name_1,name_2,geom,amenity_counts,school_count,hospital_count,pop,tot_tax_revenue,tot_current_oper_income
0,Abra,Bangued,"MULTIPOLYGON (((120.62710 17.49300, 120.62177 ...",44,2.0,NaN,48163.0,36.556294,230.577076
1,Abra,Boliney,"MULTIPOLYGON (((120.99042 17.39802, 120.98631 ...",9,NaN,NaN,3573.0,0.035633,56.689883
2,Abra,Bucay,"MULTIPOLYGON (((120.70788 17.45395, 120.70621 ...",3,NaN,NaN,17115.0,1.019565,93.647242
3,Abra,Bucloc,"MULTIPOLYGON (((120.78956 17.41699, 120.78922 ...",6,NaN,NaN,2501.0,0.129150,44.789104
4,Abra,Daguioman,"MULTIPOLYGON (((120.92917 17.41307, 120.92464 ...",10,NaN,NaN,2088.0,0.277660,51.242322


In [71]:
data['pop_density'] = data['pop'] / data.area

In [72]:
data.head(2)

,name_1,name_2,geom,amenity_counts,school_count,hospital_count,pop,tot_tax_revenue,tot_current_oper_income,pop_density
0,Abra,Bangued,"MULTIPOLYGON (((120.62710 17.49300, 120.62177 ...",44,2.0,NaN,48163.0,36.556294,230.577076,4.575788e+06
1,Abra,Boliney,"MULTIPOLYGON (((120.99042 17.39802, 120.98631 ...",9,NaN,NaN,3573.0,0.035633,56.689883,2.300790e+05


In [73]:
data.to_csv('amenities_and_finance_v1.csv')

# Data Preparation
## Merge Earthquake Dataset + Finance + Amenities

In [74]:
df_earthquake = pd.read_csv('ph_earthquake.csv')

In [75]:
df_earthquake.head(2)

,Unnamed: 0,province,municipality,Point,geom,coordinates,magnitude,time,mean_mag_2010,mean_mag_2011,...,major_2012,major_2013,major_2014,major_2015,major_2016,major_2017,major_2018,major_2019,major_2020,major_2021
0,0,Abra,Bangued,['POINT (119.8464 16.4895)'],"MULTIPOLYGON (((120.62709808 17.49300003, 120....","[119.8464, 16.4895, 37]",[5.5],['2015-11-07 19:40:13.550'],0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,1,Abra,Bucay,"['POINT (125.704 6.176)', 'POINT (123.325 14.9...","MULTIPOLYGON (((120.70787811 17.45395088, 120....","[125.704, 6.176, 122.7]","[4.6, 5.3]","['2012-07-29 03:27:50.000', '2013-05-01 05:37:...",0.0,0.0,...,1,1,0,0,0,0,0,0,0,0


In [76]:
df_earthquake.columns

Index(['Unnamed: 0', 'province', 'municipality', 'Point', 'geom',
       'coordinates', 'magnitude', 'time', 'mean_mag_2010', 'mean_mag_2011',
       'mean_mag_2012', 'mean_mag_2013', 'mean_mag_2014', 'mean_mag_2015',
       'mean_mag_2016', 'mean_mag_2017', 'mean_mag_2018', 'mean_mag_2019',
       'mean_mag_2020', 'mean_mag_2021', 'max_mag_2010', 'max_mag_2011',
       'max_mag_2012', 'max_mag_2013', 'max_mag_2014', 'max_mag_2015',
       'max_mag_2016', 'max_mag_2017', 'max_mag_2018', 'max_mag_2019',
       'max_mag_2020', 'max_mag_2021', 'min_mag_2010', 'min_mag_2011',
       'min_mag_2012', 'min_mag_2013', 'min_mag_2014', 'min_mag_2015',
       'min_mag_2016', 'min_mag_2017', 'min_mag_2018', 'min_mag_2019',
       'min_mag_2020', 'min_mag_2021', 'light_2010', 'light_2011',
       'light_2012', 'light_2013', 'light_2014', 'light_2015', 'light_2016',
       'light_2017', 'light_2018', 'light_2019', 'light_2020', 'light_2021',
       'moderate_2010', 'moderate_2011', 'moderate_2012', 

In [77]:
data.rename(columns={'name_1': 'province', 'name_2': 'municipality'}, inplace=True)

In [80]:
# merge earthquake data and amenities data
dataset = data.merge(df_earthquake, on=['province', 'municipality'], how='left')

In [81]:
dataset.head()

,province,municipality,geom_x,amenity_counts,school_count,hospital_count,pop,tot_tax_revenue,tot_current_oper_income,pop_density,...,major_2012,major_2013,major_2014,major_2015,major_2016,major_2017,major_2018,major_2019,major_2020,major_2021
0,Abra,Bangued,"MULTIPOLYGON (((120.62710 17.49300, 120.62177 ...",44,2.0,NaN,48163.0,36.556294,230.577076,4.575788e+06,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abra,Boliney,"MULTIPOLYGON (((120.99042 17.39802, 120.98631 ...",9,NaN,NaN,3573.0,0.035633,56.689883,2.300790e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abra,Bucay,"MULTIPOLYGON (((120.70788 17.45395, 120.70621 ...",3,NaN,NaN,17115.0,1.019565,93.647242,1.926863e+06,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abra,Bucloc,"MULTIPOLYGON (((120.78956 17.41699, 120.78922 ...",6,NaN,NaN,2501.0,0.129150,44.789104,5.735264e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abra,Daguioman,"MULTIPOLYGON (((120.92917 17.41307, 120.92464 ...",10,NaN,NaN,2088.0,0.277660,51.242322,2.523502e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1633 entries, 0 to 1632
Data columns (total 100 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   province                 1633 non-null   object  
 1   municipality             1633 non-null   object  
 2   geom_x                   1633 non-null   geometry
 3   amenity_counts           1633 non-null   int64   
 4   school_count             983 non-null    float64 
 5   hospital_count           357 non-null    float64 
 6   pop                      1571 non-null   float64 
 7   tot_tax_revenue          1608 non-null   float64 
 8   tot_current_oper_income  1608 non-null   float64 
 9   pop_density              1571 non-null   float64 
 10  Unnamed: 0               1406 non-null   float64 
 11  Point                    1406 non-null   object  
 12  geom_y                   1406 non-null   object  
 13  coordinates              1406 non-null   object  
 14  magnitu

In [86]:
dataset.drop(columns=['geom_y'], axis=1, inplace=True)
dataset.rename(columns={'geom_x': 'geom'}, inplace=True)

In [87]:
dataset.to_csv('dataset_for_clustering.csv')

In [117]:
dataset = pd.read_csv('dataset_for_clustering.csv')

In [118]:
# The columns of the dataset
dataset.columns

Index(['Unnamed: 0', 'province', 'municipality', 'geom', 'amenity_counts',
       'school_count', 'hospital_count', 'pop', 'tot_tax_revenue',
       'tot_current_oper_income', 'pop_density', 'Unnamed: 0.1', 'Point',
       'coordinates', 'magnitude', 'time', 'mean_mag_2010', 'mean_mag_2011',
       'mean_mag_2012', 'mean_mag_2013', 'mean_mag_2014', 'mean_mag_2015',
       'mean_mag_2016', 'mean_mag_2017', 'mean_mag_2018', 'mean_mag_2019',
       'mean_mag_2020', 'mean_mag_2021', 'max_mag_2010', 'max_mag_2011',
       'max_mag_2012', 'max_mag_2013', 'max_mag_2014', 'max_mag_2015',
       'max_mag_2016', 'max_mag_2017', 'max_mag_2018', 'max_mag_2019',
       'max_mag_2020', 'max_mag_2021', 'min_mag_2010', 'min_mag_2011',
       'min_mag_2012', 'min_mag_2013', 'min_mag_2014', 'min_mag_2015',
       'min_mag_2016', 'min_mag_2017', 'min_mag_2018', 'min_mag_2019',
       'min_mag_2020', 'min_mag_2021', 'light_2010', 'light_2011',
       'light_2012', 'light_2013', 'light_2014', 'light_2015',

In [119]:
import numpy as np

In [120]:
dataset.fillna('[0]', inplace=True)

In [121]:
dataset.isna().sum()

Unnamed: 0        0
province          0
municipality      0
geom              0
amenity_counts    0
                 ..
major_2017        0
major_2018        0
major_2019        0
major_2020        0
major_2021        0
Length: 100, dtype: int64

In [125]:
dataset['magnitude'] = dataset['magnitude'].apply(lambda x: np.array(x))

In [131]:
dataset['magnitude'] = dataset['magnitude'].apply(lambda x: ast.literal_eval(x))

In [ ]:
c = [x for x in a if x < b]

In [142]:
dataset['upper_90_mag'] = dataset['magnitude'].apply(lambda x: [i for i in x if i <= np.quantile(x, 0.90)])

In [143]:
# Final dataset to be used for analysis
dataset.to_csv('dataset_for_clustering.csv')